In [58]:
import numpy as np
import pandas as pd
import os 


df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv')
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [59]:
df.drop(columns= ['tweet_id'] , inplace= True)
df

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
...,...,...
39995,neutral,@JohnLloydTaylor
39996,love,Happy Mothers Day All my love
39997,love,Happy Mother's Day to all the mommies out ther...
39998,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


### workin on binary classification for ease thus  , 


In [60]:
df['sentiment'].unique()

array(['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger'],
      dtype=object)

In [61]:
df[df['sentiment'].isin(['happiness' , 'sadness'])]

,sentiment,content
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
6,sadness,"I should be sleep, but im not! thinking about ..."
8,sadness,@charviray Charlene my love. I miss you
9,sadness,@kelcouch I'm sorry at least it's Friday?
...,...,...
39986,happiness,going to watch boy in the striped pj's hope i ...
39987,happiness,"gave the bikes a thorough wash, degrease it an..."
39988,happiness,"had SUCH and AMAZING time last night, McFly we..."
39994,happiness,Succesfully following Tayla!!


In [62]:
final_df = df[df['sentiment'].isin(['happiness' , 'sadness'])]

In [63]:
final_df.replace({'happiness' : 1 , 'sadness'  : 0 }, inplace = True)
final_df.head()

C:\Users\Ujjwal\AppData\Local\Temp\ipykernel_400\532746371.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df.replace({'happiness' : 1 , 'sadness'  : 0 }, inplace = True)
C:\Users\Ujjwal\AppData\Local\Temp\ipykernel_400\532746371.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.replace({'happiness' : 1 , 'sadness'  : 0 }, inplace = True)


,sentiment,content
1,0,Layin n bed with a headache ughhhh...waitin o...
2,0,Funeral ceremony...gloomy friday...
6,0,"I should be sleep, but im not! thinking about ..."
8,0,@charviray Charlene my love. I miss you
9,0,@kelcouch I'm sorry at least it's Friday?


In [64]:
#train test spli
from sklearn.model_selection import train_test_split 
train_data ,  test_data  = train_test_split(final_df , test_size= 0.2 , random_state= 42)


## creating raw , train , test file 


In [65]:
%%writefile src/data_ingestion.py
#data loading

import numpy as np
import pandas as pd
import os 
from sklearn.model_selection import train_test_split 

df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv')
df.drop(columns= ['tweet_id'] , inplace= True)

final_df = df[df['sentiment'].isin(['happiness' , 'sadness'])]
final_df.replace({'happiness' : 1 , 'sadness'  : 0 }, inplace = True)

#train test spli
train_data ,  test_data  = train_test_split(final_df , test_size= 0.2 , random_state= 42)
data_path = os.path.join('data' , 'raw' )
os.makedirs(data_path, exist_ok= True)

train_data.to_csv(os.path.join(data_path , 'train.csv'))
test_data.to_csv(os.path.join(data_path , 'test.csv'))

Overwriting src/data_ingestion.py


In [66]:
%run -i src/data_ingestion.py

D:\ml-pipeline-demo\src\data_ingestion.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df.replace({'happiness' : 1 , 'sadness'  : 0 }, inplace = True)
D:\ml-pipeline-demo\src\data_ingestion.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.replace({'happiness' : 1 , 'sadness'  : 0 }, inplace = True)


In [67]:
os.makedirs(data_path, exist_ok= True)

## data ingenstion done 🙏🙏

now add to dvc state , inorder to add into dvc we need to declear few thigns

-n name of state (generally , this name will be python file name)
-d (inorder to execute currect state , which file is need to run for example
src/dataingestion will run  , raw test and train thus )

-p parameters  (configuration related parameters ) [skipped for now ]

-o (what is the  output of the currect stage )
for example what will be be output of data _ingestion.py
  answer data folders 
i.e data/raw



now write a code  in command line
dev statge add -n data-ingestion -d src/data_ingestion.py -o data/raw  [and at last 
    add coomand to run the file
    python src/data_ingestion.py

]

this coude will generate a new file called dvc.yaml

  #inorder to run the pipeline 
type dvc repro :
dvc run will automaticaly search for dvc yamal file and run the file 
repro stands for reproduce

delete the data folder and run  , you will see new data folder has been created 

## DATA transformation 

In [15]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer

In [16]:
train_data = pd.read_csv(os.path.join('data' , 'raw' , 'train.csv'))
test_data = pd.read_csv(os.path.join('data' ,'raw' , 'test.csv'))

In [17]:
train_data

,Unnamed: 0,sentiment,content
0,23531,0,&quot;My problem isn't that I miss you... 'cau...
1,8051,0,That's it? It's done already? This is one proo...
2,11499,0,I am so hungry! And there is no food for me to...
3,31288,1,Feet hurt...finally in bed...will not forget t...
4,18561,0,really ill atm
...,...,...,...
8294,21697,1,@chocolatesuze yes yes you should! Especially ...
8295,19445,0,@kickzfadayz Our boy better get it in tonight!...
8296,20216,1,tafe was actually quite good. for once
8297,3258,0,10 minutes to boarding; 14 hours to home. no w...


In [18]:
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ujjwal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ujjwal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:

def lemmatization(text):
    lemmatizer= WordNetLemmatizer()

    text = text.split()

    text=[lemmatizer.lemmatize(y) for y in text]

    return " " .join(text)

def remove_stop_words(text):
    stop_words = set(stopwords.words("english"))
    Text=[i for i in str(text).split() if i not in stop_words]
    return " ".join(Text)

def removing_numbers(text):
    text=''.join([i for i in text if not i.isdigit()])
    return text

def lower_case(text):

    text = text.split()

    text=[y.lower() for y in text]

    return " " .join(text)

def removing_punctuations(text):
    ## Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛',"", )

    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text =  " ".join(text.split())
    return text.strip()

def removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_small_sentences(df):
    for i in range(len(df)):
        if len(df.text.iloc[i].split()) < 3:
            df.text.iloc[i] = np.nan

def normalize_text(df):
    df.content=df.content.apply(lambda content : lower_case(content))
    df.content=df.content.apply(lambda content : remove_stop_words(content))
    df.content=df.content.apply(lambda content : removing_numbers(content))
    df.content=df.content.apply(lambda content : removing_punctuations(content))
    df.content=df.content.apply(lambda content : removing_urls(content))
    df.content=df.content.apply(lambda content : lemmatization(content))
    return df

def normalized_sentence(sentence):
    sentence= lower_case(sentence)
    sentence= remove_stop_words(sentence)
    sentence= removing_numbers(sentence)
    sentence= removing_punctuations(sentence)
    sentence= removing_urls(sentence)
    sentence= lemmatization(sentence)
    return sentence

In [20]:
normalized_sentence(f"Hellow my nam is 1232 ujjwal basnet {534534}")

'hellow nam ujjwal basnet'

In [21]:
train_processed_data = normalize_text(train_data)
test_processed_data = normalize_text(test_data)

In [22]:
path = os.path.join('data' , 'processed')
os.makedirs(path, exist_ok= True)
train_processed_data.to_csv(os.path.join(path ,'train_processed.csv' ))
test_processed_data.to_csv(os.path.join(path ,'test_processed.csv' ))


## writing data_preprocessing  file now

In [23]:
%%writefile src/data_preprocessing.py 
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
train_data = pd.read_csv(os.path.join('data' , 'raw' , 'train.csv'))
test_data = pd.read_csv(os.path.join('data' ,'raw' , 'test.csv'))
nltk.download('wordnet')
nltk.download('stopwords')


def lemmatization(text):
    lemmatizer= WordNetLemmatizer()

    text = text.split()

    text=[lemmatizer.lemmatize(y) for y in text]

    return " " .join(text)

def remove_stop_words(text):
    stop_words = set(stopwords.words("english"))
    Text=[i for i in str(text).split() if i not in stop_words]
    return " ".join(Text)

def removing_numbers(text):
    text=''.join([i for i in text if not i.isdigit()])
    return text

def lower_case(text):

    text = text.split()

    text=[y.lower() for y in text]

    return " " .join(text)

def removing_punctuations(text):
    ## Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛',"", )

    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text =  " ".join(text.split())
    return text.strip()

def removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_small_sentences(df):
    for i in range(len(df)):
        if len(df.text.iloc[i].split()) < 3:
            df.text.iloc[i] = np.nan

def normalize_text(df):
    df.content=df.content.apply(lambda content : lower_case(content))
    df.content=df.content.apply(lambda content : remove_stop_words(content))
    df.content=df.content.apply(lambda content : removing_numbers(content))
    df.content=df.content.apply(lambda content : removing_punctuations(content))
    df.content=df.content.apply(lambda content : removing_urls(content))
    df.content=df.content.apply(lambda content : lemmatization(content))
    return df

def normalized_sentence(sentence):
    sentence= lower_case(sentence)
    sentence= remove_stop_words(sentence)
    sentence= removing_numbers(sentence)
    sentence= removing_punctuations(sentence)
    sentence= removing_urls(sentence)
    sentence= lemmatization(sentence)
    return sentence


train_processed_data = normalize_text(train_data)
test_processed_data = normalize_text(test_data)

path = os.path.join('data' , 'processed')
os.makedirs(path, exist_ok= True)
train_processed_data.to_csv(os.path.join(path ,'train_processed.csv' ))
test_processed_data.to_csv(os.path.join(path ,'test_processed.csv' ))

Overwriting src/data_preprocessing.py


In [24]:
#testing
%run src/data_preprocessing.py

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ujjwal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ujjwal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### row run 
###  dvc stage add -n data_preprocessing -d src/data_preprocessing.py -o data/processed python src/data_preproecssing.py 
### to add on dvc

### also you can use dvc dag to  visulalize


but this is wrong  ,we should also give  , previous output as another dependency
i.e -d data/raw
so our command should be 
dvc stage add -n data_preprocessing -d src/data_preprocessing.py -d data/raw-o data/processed python src/data_preproecssing.py 

 dvc add stage add -n data_preprocessing -d src/data_preprocessing -d data/raw -0 data/processed python src/data_preprocessing.py

### 3) Feature Engeneering 

In [25]:
#imports 
import numpy as np
import os 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer



In [26]:
#featch the data from data/preocessed
train_data = pd.read_csv(os.path.join('data' , 'processed' , 'train_processed.csv'))
test_data = pd.read_csv(os.path.join('data' ,'processed' , 'test_processed.csv'))

In [73]:
#remove missing values
train_data.fillna('', inplace = True)
test_data.fillna('',inplace = True)

In [75]:
train_data[['sentiment' , 'content']]

,sentiment,content
23531,0,&quot;My problem isn't that I miss you... 'cau...
8051,0,That's it? It's done already? This is one proo...
11499,0,I am so hungry! And there is no food for me to...
31288,1,Feet hurt...finally in bed...will not forget t...
18561,0,really ill atm
...,...,...
21697,1,@chocolatesuze yes yes you should! Especially ...
19445,0,@kickzfadayz Our boy better get it in tonight!...
20216,1,tafe was actually quite good. for once
3258,0,10 minutes to boarding; 14 hours to home. no w...


In [76]:
train_data.shape[1]

2

In [77]:
#apply bow 
if train_data.shape[1] > 2 :
    train_data = train_data[['sentiment' , 'content']]
    test_data = train_data[['sentiment' , 'content']]


X_train = train_data['content'].values
y_train = train_data['sentiment'].values
X_test = test_data['content'].values
y_test = test_data['sentiment'].values

In [78]:
train_data

,sentiment,content
23531,0,&quot;My problem isn't that I miss you... 'cau...
8051,0,That's it? It's done already? This is one proo...
11499,0,I am so hungry! And there is no food for me to...
31288,1,Feet hurt...finally in bed...will not forget t...
18561,0,really ill atm
...,...,...
21697,1,@chocolatesuze yes yes you should! Especially ...
19445,0,@kickzfadayz Our boy better get it in tonight!...
20216,1,tafe was actually quite good. for once
3258,0,10 minutes to boarding; 14 hours to home. no w...


In [79]:
# Apply Bag of Words (CountVectorizer)
vectorizer = CountVectorizer()

# Fit the vectorizer on the training data and transform it
X_train_bow = vectorizer.fit_transform(X_train)

# Transform the test data using the same vectorizer
X_test_bow = vectorizer.transform(X_test)

In [80]:
#fit the vetorizer on the trainning data and transofrm it 
vectorizer = CountVectorizer(max_features= 500)
X_train_bow = vectorizer.fit_transform(X_train)

#transorm the test data using the  same vector
X_test_bow = vectorizer.fit_transform(X_test)

In [85]:
y_train

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [86]:
train_df = pd.DataFrame(X_train_bow.toarray())
train_df['label'] = y_train
test_df = pd.DataFrame(X_test_bow.toarray())
test_df['label'] = y_test

In [ ]:
train_b

In [88]:
#store 
path = os.path.join('data' , 'features')
os.makedirs(path,exist_ok= True)

train_df.to_csv(os.path.join(path  , 'train_bow.csv'))
test_df.to_csv(os.path.join(path , 'test_bow.csv'))



## write file now

In [95]:
%%writefile src/feature_engineering.py
import numpy as np
import os 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer

#featch the data from data/preocessed
train_data = pd.read_csv(os.path.join('data' , 'processed' , 'train_processed.csv'))
test_data = pd.read_csv(os.path.join('data' ,'processed' , 'test_processed.csv'))

#remove missing values
train_data.fillna('', inplace = True)
test_data.fillna('',inplace = True)


if train_data.shape[1] > 2 :
    train_data = train_data[['sentiment' , 'content']]
    test_data = train_data[['sentiment' , 'content']]


X_train = train_data['content'].values
y_train = train_data['sentiment'].values
X_test = test_data['content'].values
y_test = test_data['sentiment'].values

# Apply Bag of Words (CountVectorizer)
vectorizer = CountVectorizer(max_features= 500)

# Fit the vectorizer on the training data and transform it
X_train_bow = vectorizer.fit_transform(X_train)

# Transform the test data using the same vectorizer
X_test_bow = vectorizer.transform(X_test)


train_df = pd.DataFrame(X_train_bow.toarray()) 
train_df['label'] = y_train
test_df = pd.DataFrame(X_train_bow.toarray())
test_df['label']=y_test

#store 
path = os.path.join('data' , 'features')
os.makedirs(path,exist_ok= True)



train_df.to_csv(os.path.join(path  , 'train_bow.csv'))
test_df.to_csv(os.path.join(path , 'test_bow.csv'))



Overwriting src/feature_engineering.py


dvc stage add -n feature_engeenering -d src/feature_engineering -d data/processed 

-o data/features python src/feature_engineering.py

In [92]:
%run src/feature_engineering.py

## model building

In [112]:
import numpy as np 
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier 
import os
import pickle

In [114]:
#fetch 
train_data = pd.read_csv(os.path.join('data','features' , 'train_bow.csv'))
X_train = train_data.iloc[: , 0:-1].values 
y_train = train_data.iloc[:,-1].values
X_train.shape , y_train.shape

((8299, 501), (8299,))

In [107]:
X_train.shape

(8299, 501)

In [108]:
y_train.shape

(8299,)

In [113]:
#
clf = GradientBoostingClassifier(n_estimators= 50)
clf = clf.fit(X_train , y_train)

#save
pickle.dump(clf , open('model.pkl' , 'wb'))

## writting model file

In [115]:
%%writefile src/model_building.py
import numpy as np 
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier 
import os
import pickle

#fetch 
train_data =  pd.read_csv(os.path.join('data' , 'features' , 'train_bow.csv'))
x_train = train_data.iloc[: , 0:-1]
y_train = train_data.iloc[: , -1]

#define model and train
clf = GradientBoostingClassifier(n_estimators= 50)
clf.fit(x_train , y_train)

#save 
pickle.dump(clf,open('model.pkl' , 'wb'))

Writing src/model_building.py


In [116]:
%run src/model_building.py

# model evaluation 

In [140]:
%%writefile src/model_evaluation.py 
import numpy as np
import pandas as pd 
import pickle 
import json 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score  , recall_score , roc_auc_score 


#fetch
clf = pickle.load(open('model.pkl','rb'))
test_data = pd.read_csv('data/features/test_bow.csv')
x_test = test_data.iloc[: , 0:-1].values
y_test = test_data.iloc[: , -1].values

y_pred = clf.predict(x_test)
y_pred_proba = clf.predict_proba(x_test)[: , 1]


#calcuate evaluation metrics 
accuracy = accuracy_score(y_test , y_pred)
precision  = precision_score(y_test, y_pred) 
recall = recall_score(y_test , y_pred)
auc = roc_auc_score(y_test , y_pred_proba)

metrics_dict = {
    'accuracy' : accuracy , 
    'precision' : precision , 
    'recall' : recall , 
    'auc' : auc}

with open('metrics.json' , 'w') as file :
    json.dump(metrics_dict ,file , indent = 4)




Overwriting src/model_evaluation.py


In [141]:
%run src/model_evaluation.py

d:\software\miniconda\envs\mlenv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
d:\software\miniconda\envs\mlenv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


## we are saving metrics , as output so that  , given metrics can show and plot in coomand line so 
##  --metrics metrics.json ....
 dvc stage add -n model_evaluation -d src/model_evaluation.py -d data/features -d model.pkl --metrics metrics.json python src/model_evaluation.py 

 
 


## dvc metrics show 